# Download and extract waterways from OpenStreetMap (OSM)
Here are two ways to download and extract a network consists of rivers and streams from the OSM (* .pbf) data.

## 1. Using pyrosm package
See this package on [GitHub](https://github.com/HTenkanen/pyrosm) for more detail.

In [1]:
from pyrosm import OSM, get_data
from fiona.crs import from_epsg
import time

In [2]:
my_place = 'liechtenstein'

start_time = time.time()
print(f"\nStart downloading OSM data for " + my_place.title() + "...")
fp = get_data(my_place)
print(f"Downloading finished,"
      f" lasted {round(time.time() - start_time, 0)} seconds.")
osm = OSM(fp)


Start downloading OSM data for Liechtenstein...
Downloaded Protobuf data 'liechtenstein-latest.osm.pbf' (2.54 MB) to:
'C:\Users\zhouy\AppData\Local\Temp\pyrosm\liechtenstein-latest.osm.pbf'


In [3]:
# Custom filter
waterways = ['river', 'stream']

In [4]:
print(f"\nStart parsing custom lines...")
my_waterways = osm.get_data_by_custom_criteria(custom_filter={
    'waterway': waterways},
    filter_type='keep',
    keep_nodes=False,
    keep_ways=True,
    keep_relations=True)
print(f"Parsing lines finished, "
      f"lasted {round(time.time() - start_time, 0)} seconds.")
print(f"Number of lines parsed: {len(my_waterways)}")


Start parsing custom lines...
Parsing lines finished, lasted 2.0 seconds.
Number of lines parsed: 348


C:\Users\zhouy\Anaconda3\envs\geo_env\lib\site-packages\pyrosm\user_defined.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  osm_keys=osm_keys,


Information to this warning see: https://github.com/HTenkanen/pyrosm/issues/50

In [5]:
# The result may contain some polygons (not in this case), filter out these polygons if necessary
my_waterways['geom_type'] = my_waterways.geometry.geom_type
print(my_waterways['geom_type'].unique())

my_waterways = my_waterways[my_waterways['geom_type'] != 'Polygon']

['LineString' 'MultiLineString']


See https://github.com/HTenkanen/pyrosm/issues/67 for the polygon issue.

In [6]:
# Count the number of each waterway type
type_waterways = my_waterways.groupby('waterway')
print(type_waterways.count())

           id  timestamp  version  tags  geometry  osm_type  changeset  \
waterway                                                                 
river       4          4        4     3         4         4          1   
stream    344        344      344   208       344       344          0   

          geom_type  
waterway             
river             4  
stream          344  


In [7]:
# Re-project the network before simplification
my_waterways = my_waterways.to_crs(epsg=25832)

In [8]:
# # Now you can use shapely_filter.py for further simplification (see README.md) or just export as a GeoJSON file or shapefile
# multi_line = [shape(feature) for feature in my_waterways['geometry']]
# multi_line_union = unary_union(multi_line)
# multi_line_merge = linemerge(multi_line_union)

# my_multi_line = LineFilter(multi_line_merge)
# multi_line_filtered = my_multi_line.simplify_line()

## 2. Using esy-osmfilter package
See this package on [GitLab](https://gitlab.com/dlr-ve-esy/esy-osmfilter) for more detail. The package assuming the * .pbf file is already [downloaded](http://download.geofabrik.de/europe/liechtenstein.html).

In [9]:
import os
from esy.osmfilter import Node, Way, Relation
from esy.osmfilter import run_filter
from esy.osmfilter import export_geojson

In [10]:
# You need to download the *.pbf file first, and put it in your working directory
# Define input and output
PBF_inputfile = os.path.join(os.getcwd(), 'liechtenstein-latest.osm.pbf')
JSON_outputfile = os.path.join(os.getcwd(), 'liechtenstein_river_stream.json')

# Custom filter
prefilter = {Node: {}, Way: {'waterway': ['river', 'stream', ], }, Relation: {}}
blackfilter = [("waterway", "riverbank"), ]  # we don't want 'riverbank' in our results
whitefilter = [(("waterway", "river"),), (("waterway", "stream"),), ]

# Run osmfilter
[Data, Elements] = run_filter('liechtenstein-latest-river-stream',
                       PBF_inputfile,
                       JSON_outputfile,
                       prefilter,
                       whitefilter,
                       blackfilter,
                       NewPreFilterData=True,
                       CreateElements=True,
                       LoadElements=False,
                       verbose=False,
                       multiprocess=True)

print(f"Number of lines after filtering: "
      + str(len(Elements['liechtenstein-latest-river-stream']['Way'])))

Number of lines after filtering: 349


The numbers of filtered lines from the two methods are different, however, after union and merge, the networks have the same lines (402 lines)

In [11]:
# Export to GeoJSON file
export_geojson(Elements['liechtenstein-latest-river-stream']['Way'], Data,
               filename='liechtenstein_river_stream.geojson', jsontype='Line')

Remember to re-project the network before simplification using shapely_filter.py module.